In [1]:
# Import required packages/libraries:
import os
import sys
from pyspark.sql.types import DoubleType
from pyspark.sql.functions import isnan, when, count, col

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
29,application_1606590675617_0030,pyspark,idle,,,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
# Define sql context:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
# Read the entire dataset from S3 as a Spark dataframe:
df = spark.read.format("csv").option("inferSchema", "true").option("header","true").load("s3://projectmlldsongs/processed_songs/*.csv")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
print(df.count())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

505114

In [5]:
# Use Regex to filter numeric entries in target column i.e. 'song_hotttnesss':
expr = "(^[+-]?([0-9]*[.])?[0-9]+)"
df = (df.filter(df.song_hotttnesss.rlike(expr)))
df = df.filter(df.song_hotttnesss > 0.0)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
# Seperate and typecast(to double) features having numeric entries:
numeric_columns=['artist_familiarity',
                 'artist_hotttnesss',
                 'duration',
                 'end_of_fade_in',
                 'energy',
                 'mode',
                 'key',
                 'key_confidence',
                 'loudness',
                 'song_hotttnesss',
                 'tempo',
                 'time_signature',
                 'time_signature_confidence',
                 'year']

for c in numeric_columns:
    df=df.withColumn(c,df[c].cast(DoubleType()))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
# Drop rows with null/NA values and duplicates:
df = df.na.drop()
df=df.drop_duplicates()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
# Select columns for the regression model:
select_df = df.select([ 'artist_familiarity','duration','loudness','key_confidence','key','end_of_fade_in',
                       'time_signature_confidence','tempo','mode','song_hotttnesss'])

# Save full dataset in S3 for future use:
select_df.write.parquet("s3://projectmlldsongs/songs_data_split_v2/type=full/")


# Split dataset into train and validation sets:
train_df,test_df_model =select_df.randomSplit([0.75, 0.25])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
# Save train and test dataframes in S3 for future use:
train_df.write.parquet("s3://projectmlldsongs/songs_data_split_v2/type=train/")
test_df_model.write.parquet("s3://projectmlldsongs/songs_data_split_v2/type=test/")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…